In [16]:
!pip install keras_tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 9.2 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load dataset


In [2]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

df_train = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "zalando-research/fashionmnist",
    "fashion-mnist_train.csv"
)
df_test = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "zalando-research/fashionmnist",
    "fashion-mnist_test.csv"
)

100%|██████████| 33.6M/33.6M [00:00<00:00, 115MB/s] 

Extracting zip of fashion-mnist_train.csv...


In [3]:
df_train.head(5)

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df_test.head(5)

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Number of featuers and observations in train data  :

In [5]:
df_train.shape

(60000, 785)

Number of featuers and observations in test data  :

In [6]:
df_test.shape

(10000, 785)

check duplicated values in train and test data :

In [7]:
print(f'Number of duplicated observation in train data = {df_train.duplicated().sum()}')
print(f'Number of duplicated observation in test data = {df_test.duplicated().sum()}')

Number of duplicated observation in train data = 43
Number of duplicated observation in test data = 1


drop duplicated values :

In [8]:
df_train.drop_duplicates(inplace=True)
df_test.drop_duplicates(inplace=True)

check missing values in the data :

In [9]:
print(f'Number of missing values in train data  = {df_train.isna().sum().sum()}')
print(f'Number of missing values in train data  = {df_test.isna().sum().sum()}')

Number of missing values in train data  = 0
Number of missing values in train data  = 0


In [10]:
X = df_train.drop(columns="label").astype("float32") / 255.0
X.head(10)


,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.019608,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.117647,0.168627,0.000000,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.003922,0.007843,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.011765,0.000000,0.000000,0.000000,0.000000,0.003922,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
5,0.0,0.0,0.0,0.019608,0.015686,0.019608,0.019608,0.011765,0.019608,0.023529,...,0.027451,0.031373,0.027451,0.015686,0.011765,0.027451,0.019608,0.0,0.0,0.0
6,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.054902,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
7,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
8,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.011765,0.007843,0.000000,0.000000,...,0.003922,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
9,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.796078,0.839216,0.650980,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0


In [11]:
y = df_train["label"]
y.head(10)

,label
0,2
1,9
2,6
3,0
4,3
5,4
6,4
7,5
8,4
9,8


In [12]:
X_test = df_test.drop(columns="label").astype("float32") / 255.0
X_test.head(10)

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.035294,0.031373,0.000000,...,0.403922,0.341176,0.219608,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.054902,0.207843,0.388235,0.066667,...,0.000000,0.000000,0.000000,0.000000,0.247059,0.207843,0.121569,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.631373,...,0.537255,0.494118,0.549020,0.000000,0.521569,0.878431,0.870588,0.219608,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.172549,0.411765,0.172549,0.039216,0.000000,...,0.411765,0.250980,0.117647,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.003922,0.000000,0.000000,...,0.682353,0.533333,0.607843,0.121569,0.000000,0.003922,0.000000,0.000000,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.223529,0.274510,0.109804,0.000000,0.007843,0.000000,0.000000,0.000000,0.0,0.0


In [13]:
y_test = df_test["label"]
y_test.head(10)

,label
0,0
1,1
2,2
3,2
4,3
5,2
6,8
7,6
8,5
9,0


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Build model function for Keras Tuner


In [18]:
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers

Model Arch

In [19]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train.shape[1],)))

    # Hidden layers
    for i in range(hp.Int("n_hidden_layers", 1, 3)):
        model.add(layers.Dense(
            units=hp.Choice("neurons", [32, 64, 128]),
            activation=hp.Choice("activation", ["relu", "tanh", "elu"])
        ))

        if hp.Choice("dropout", [0.0, 0.3, 0.5]) > 0:
            model.add(layers.Dropout(hp.Choice("dropout", [0.0, 0.3, 0.5])))

    model.add(layers.Dense(10, activation="softmax"))

    # Optimizer choice
    optimizer_name = hp.Choice("optimizer", ["adam", "sgd", "rmsprop"])
    lr = hp.Choice("learning_rate", [1e-2, 1e-3])

    if optimizer_name == "adam":
        optimizer = keras.optimizers.Adam(learning_rate=lr)
    elif optimizer_name == "sgd":
        optimizer = keras.optimizers.SGD(learning_rate=lr, momentum=0.9, nesterov=True)
    else:
        optimizer = keras.optimizers.RMSprop(learning_rate=lr)

    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model


# Run Keras Tuner


In [24]:
tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=5,
    executions_per_trial=1,
    directory="my_tuner",
    project_name="fashion_mnist"
)

Reloading Tuner from my_tuner/fashion_mnist/tuner0.json


In [25]:
tuner.search(
    X_train, y_train,
    epochs=10,                # quick search
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=0
)

# Get best model


In [26]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best Hyperparameters:")
print(best_hps.values)

best_model = tuner.get_best_models(num_models=1)[0]

Best Hyperparameters:
{'n_hidden_layers': 1, 'neurons': 64, 'activation': 'tanh', 'dropout': 0.3, 'optimizer': 'rmsprop', 'learning_rate': 0.001}


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [29]:
from tensorflow.keras.callbacks import EarlyStopping


In [30]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=2,
    restore_best_weights=True
)

history = best_model.fit(
    X_train, y_train,
    epochs=15,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stop]
)

Epoch 1/15
1499/1499 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8905 - loss: 0.3063 - val_accuracy: 0.8843 - val_loss: 0.3359
Epoch 2/15
1499/1499 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8924 - loss: 0.3023 - val_accuracy: 0.8827 - val_loss: 0.3357
Epoch 3/15
1499/1499 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8960 - loss: 0.2940 - val_accuracy: 0.8795 - val_loss: 0.3409
Epoch 4/15
1499/1499 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8947 - loss: 0.2982 - val_accuracy: 0.8816 - val_loss: 0.3377


In [32]:
test_loss, test_acc = best_model.evaluate(X_test, y_test, verbose=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8782 - loss: 0.3368


In [33]:
print("Test Accuracy:", test_acc)


Test Accuracy: 0.8823882341384888


In [34]:
from sklearn.metrics import classification_report, confusion_matrix

In [35]:
y_pred = np.argmax(best_model.predict(X_test), axis=1)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.87      0.83       999
           1       0.97      0.98      0.98      1000
           2       0.83      0.78      0.81      1000
           3       0.92      0.89      0.90      1000
           4       0.77      0.89      0.82      1000
           5       0.96      0.93      0.94      1000
           6       0.75      0.62      0.68      1000
           7       0.92      0.94      0.93      1000
           8       0.96      0.97      0.97      1000
           9       0.94      0.96      0.95      1000

    accuracy                           0.88      9999
   macro avg       0.88      0.88      0.88      9999
weighted avg       0.88      0.88      0.88      9999


Confusion Matrix:
 [[867   2   8  22   1   2  85   0  12   0]
 [  1 984   2   6   1   2   4   0   0   0]
 [ 17   0 782  11 125   0  62   0   3   0]
 [ 29  23  10 888  35   2 